# General Imports

In [ ]:
import numpy as np
from matplotlib import animation, rc
import matplotlib.pyplot as plt

import tools.utils as utils
from IPython.display import display, Math, Latex, Markdown, HTML

%matplotlib inline


In [ ]:
def print_beliefs(beliefs:np.array)->None:
    """
    Function to format our beliefs

    Args:
        beliefs (np.array): beliefs to be printed
    """
    print("position\tbelief")
    for position, belief in enumerate(beliefs):
        print(f"{position}\t\t{belief[0]:.3f}")

def normpdf(x:np.array, mean:float, std:float)->np.array:
    """
    Implemented Gaussian to calculate the probabilities

    Args:
        x (np.array): points to calculate the gaussian
        mean (float): center of our probability distribution
        std (float): standard deviation of our probability distribution

    Returns:
        np.array: array with the calculated gaussians
    """
    return (1/np.sqrt(2*np.pi))*np.exp(-0.5*np.power((x-mean)/std,2))

## Initialize priors

In [ ]:
def initialize_priors(map_size:int, landmark_positions:np.array, position_stdev:float)->np.array:
    """
    Function that initialized the beliefs the the car is in a given position

    Args:
        map_size (int): Size of the map
        landmark_positions (np.array): Positions in the map where ladmarks are located
        position_stdev (float): Standard deviation of the position of the landmarks

    Returns:
        np.array: Array with the initialized belief of the car being in a given position
    """
    priors = np.zeros((map_size,1))
    norm_term = len(landmark_positions)*(position_stdev*2+1)
    priors[landmark_positions] += 1.0/norm_term
    priors[landmark_positions+1] += 1.0/norm_term
    priors[landmark_positions-1] += 1.0/norm_term

    return priors

map_size = 25
landmark_positions = np.asarray([3,9,14,23])
position_stdev = 1

priors = initialize_priors(map_size, landmark_positions, position_stdev)
print_beliefs(priors)

## Motion Model

In [ ]:
def motion_model(pseudo_position:float, movement:float, priors:np.array, map_size:int, control_std:float)->float:
    """
    Implementation of the motion model in python. As discussed in the lesson, the probability of the car being in a position given
    a movement and priors. The probability is modeled as a Normal distribution 
    
    N(distance from a given position and a next position, movement, cotrol_std)

    Args:
        pseudo_position (float): Position where we want to calculate the probability 
                                 of landing in this position given a movement, a prior belief and a control standard deviation
        movement (float): Control input
        priors (np.array): Prior beliefs of the position of the car
        map_size (int): Size of the map
        control_std (float): Control standard deviation

    Returns:
        float: the probability of landing in a pseudo position given a movement, a prior belief and a control standard deviation
    """
    position_prob = 0
    distances = pseudo_position-np.arange(map_size)
    probs = normpdf(distances, movement, control_std)
    position_prob = probs[None,:]@priors
    return position_prob[0,0]

control_stdev = 1
movement = 1
pseudo_position = 5

position_prob = motion_model(pseudo_position, movement, priors, map_size, control_stdev)
print(position_prob)

## Pseudo Range Estimator

In [ ]:
def pseudo_range_estimator(landmark_positions:np.array, pseudo_position:float)->np.array:
    """
    Function that estimates the pseudo ranges given a position. Remember that a pseudo range
    is the distance the car would observe if its state is pseudo position. 

    Args:
        landmark_positions (np.array): 
        pseudo_position (float): Positions in the map where ladmarks are located

    Returns:
        np.array: estimated pseudo ranges
    """
    pseudo_ranges = landmark_positions - pseudo_position
    return np.sort(pseudo_ranges[pseudo_ranges>0])

pseudo_ranges = pseudo_range_estimator(landmark_positions, pseudo_position)

print(f"{pseudo_position = }")
print(f"landmark positions = {landmark_positions}")
print(f"pseudo ranges = {pseudo_ranges}")


## Observation Model

In [ ]:
def observation_model(observations:np.array, pseudo_ranges:np.array, observation_stdev:float)->float:
    """
    Implementation of the observation model. The observation model is the probability of having an observation given
    that we are in a given position that yields the pseudo ranges.

    Args:
        observations (np.array): Observations obtained by our sensor
        pseudo_ranges (np.array): Pseudo ranges obtained in a given pseudo position
        observation_stdev (float): Standard deviation of the noise of our sensor

    Returns:
        (float): Calculated probability
    """
    distance_prob = 1

    for idx, observation in enumerate(observations):
        pseudo_range_min = 0
        if idx < len(pseudo_ranges):
            pseudo_range_min = pseudo_ranges[idx]
        else:
            pseudo_range_min = np.inf
        
        distance_prob = distance_prob*normpdf(observation, pseudo_range_min, observation_stdev)

    return distance_prob

observations = np.asarray([1,7,12,21])
observation_stdev = 1
observation_prob = observation_model(observations, pseudo_ranges, observation_stdev)

print(f"Observations = {observations}")
print(f"pseudo ranges = np.asarray{pseudo_ranges}")
print(f"Observation probability = {observation_prob}")

## All together

In [ ]:
# set standard deviation of control
control_std = 1

# set standard deviation of position
position_std = 1

# meters vehicle moves per time step
movement= 1

# set observation standard deviation
observation_std = 1

# number of x positions on map
map_size = 25

# set distance max
distance_max = map_size

# define landmarks
landmark_positions= np.asarray([3, 9, 14, 23])

# define observations vector, each inner vector represents a set
# of observations for a time step

sensor_obs = [np.asarray([1,7,12,21]), 
              np.asarray([0,6,11,20]), 
              np.asarray([5,10,19]),
              np.asarray([4,9,18]), 
              np.asarray([3,8,17]), 
              np.asarray([2,7,16]), 
              np.asarray([1,6,15]),
              np.asarray([0,5,14]), 
              np.asarray([4,13]), 
              np.asarray([3,12]), 
              np.asarray([2,11]), 
              np.asarray([1,10]),
              np.asarray([0,9]), 
              np.asarray([8]), 
              np.asarray([7]), 
              np.asarray([6]), 
              np.asarray([5]), 
              np.asarray([4]), 
              np.asarray([3]), 
              np.asarray([2]),
              np.asarray([1]), 
              np.asarray([0]), 
              np.asarray([]), 
              np.asarray([]), 
              np.asarray([])]
   


In [ ]:
priors = initialize_priors(map_size, landmark_positions, position_stdev)
posteriors = np.zeros((map_size,1))
print_beliefs(priors)
# Cycle through timesteps
for t, observations in enumerate(sensor_obs):
    
    if len(observations) < 1:
        observations = np.asarray([distance_max])
    
    # step through each pseudo position x (i)
    for pseudo_position in range(map_size):

        # Calculate the motion model probability for each x position
        motion_prob = motion_model(pseudo_position, movement, priors, map_size, control_std)
        
        # Calculate the pseudo ranges
        pseudo_ranges = pseudo_range_estimator(landmark_positions, pseudo_position)

        # Get the observation probability
        observation_prob = observation_model(observations, pseudo_ranges, observation_std)

        posteriors[pseudo_position, 0] = motion_prob*observation_prob

    posteriors = posteriors/np.sum(posteriors)

    print(f"Timestep {t}")
    print(f"Observations {observations}")
    print_beliefs(posteriors)

    priors = posteriors.copy()